In [2]:
import duckdb
import pandas as pd
from deltalake import DeltaTable
import boto3
import subprocess
import sys

# ============================================================
# Cấu hình kết nối MinIO (qua DuckDB)
# ============================================================
print("🔗 Đang cấu hình kết nối tới MinIO...")

duckdb.sql("""
INSTALL httpfs;
LOAD httpfs;

SET s3_region='us-east-1';
SET s3_url_style='path';
SET s3_endpoint='127.0.0.1:9000';
SET s3_use_ssl=false;
SET s3_access_key_id='minioadmin';
SET s3_secret_access_key='minioadmin';
""")

print("✅ Cấu hình MinIO thành công.\n")

# ============================================================
# Đường dẫn Delta Lake
# ============================================================
bucket = "real-estate"
prefix = "lake/bronze/property/"
endpoint_url = "http://127.0.0.1:9000"

# ============================================================
# Liệt kê danh sách file .parquet trong MinIO (không dùng ký tự `*`)
# ============================================================
print("📁 Đang liệt kê các file Parquet thực tế...")

s3 = boto3.client(
    "s3",
    endpoint_url=endpoint_url,
    aws_access_key_id="minioadmin",
    aws_secret_access_key="minioadmin",
    region_name="us-east-1",
)

objects = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
parquet_files = [
    f"{endpoint_url}/{bucket}/{obj['Key']}"
    for obj in objects.get("Contents", [])
    if obj["Key"].endswith(".parquet")
]

if not parquet_files:
    print("⚠️ Không tìm thấy file Parquet nào trong MinIO.")
    exit()

print(f"✅ Tìm thấy {len(parquet_files)} file Parquet:")
for f in parquet_files:
    print("-", f)
print()

🔗 Đang cấu hình kết nối tới MinIO...
✅ Cấu hình MinIO thành công.

📁 Đang liệt kê các file Parquet thực tế...
✅ Tìm thấy 7 file Parquet:
- http://127.0.0.1:9000/real-estate/lake/bronze/property/part-00001-030e4554-dac3-4834-8352-d99112ff55e1-c000.snappy.parquet
- http://127.0.0.1:9000/real-estate/lake/bronze/property/part-00001-44edc1b0-9211-4877-b1be-61cf6b63d9e8-c000.snappy.parquet
- http://127.0.0.1:9000/real-estate/lake/bronze/property/part-00001-5c135b9b-f6e6-4297-8de7-c4d3f3d9fa19-c000.snappy.parquet
- http://127.0.0.1:9000/real-estate/lake/bronze/property/part-00001-b09c2a12-a261-4685-89ba-32f1c2f39e4e-c000.snappy.parquet
- http://127.0.0.1:9000/real-estate/lake/bronze/property/part-00001-b0e96fc2-ade9-4cc2-bcc0-211f64fab680-c000.snappy.parquet
- http://127.0.0.1:9000/real-estate/lake/bronze/property/part-00001-b7cfb0b8-5233-40b1-a8d9-22e4f51219a6-c000.snappy.parquet
- http://127.0.0.1:9000/real-estate/lake/bronze/property/part-00001-c682ef09-4a2a-48cd-9e76-8359837f8128-c000.sna

In [3]:
# ============================================================
# Đọc metadata Delta Lake
# ============================================================
print("📁 Đang đọc metadata Delta Lake...")

dt = DeltaTable(
    "s3a://real-estate/lake/bronze/property",
    storage_options={
        "AWS_ACCESS_KEY_ID": "minioadmin",
        "AWS_SECRET_ACCESS_KEY": "minioadmin",
        "AWS_ENDPOINT_URL": endpoint_url,
        "AWS_ALLOW_HTTP": "true"
    }
)

print(f"📦 Version hiện tại của Delta Lake: {dt.version()}")
print("📄 Các file parquet hiện có:")
print(dt.files())
print()



📁 Đang đọc metadata Delta Lake...
📦 Version hiện tại của Delta Lake: 5
📄 Các file parquet hiện có:
['part-00001-b7cfb0b8-5233-40b1-a8d9-22e4f51219a6-c000.snappy.parquet', 'part-00001-5c135b9b-f6e6-4297-8de7-c4d3f3d9fa19-c000.snappy.parquet', 'part-00001-44edc1b0-9211-4877-b1be-61cf6b63d9e8-c000.snappy.parquet', 'part-00001-b09c2a12-a261-4685-89ba-32f1c2f39e4e-c000.snappy.parquet', 'part-00001-b0e96fc2-ade9-4cc2-bcc0-211f64fab680-c000.snappy.parquet']



In [4]:
# ============================================================
# Truy vấn DuckDB với danh sách file cụ thể
# ============================================================
print("\n🔍 Đang truy vấn DuckDB với danh sách file cụ thể...")

try:
    # Chúng ta dùng '?' để truyền biến Python (params) vào SQL
    # Cần dấu ngoặc kép "..." quanh tên cột vì nó có chữ hoa/thường
    result_df = duckdb.sql(
        'SELECT "propertyDetails_propertyId" FROM read_parquet(?)',
        params=[parquet_files]  
    ).df()

    print("✅ Truy vấn thành công!")
    print(f"Tìm thấy {len(result_df)} dòng.")
    print("\n--- 5 dòng đầu tiên ---")
    print(result_df.head())

except Exception as e:
    print(f"❌ Lỗi khi truy vấn DuckDB: {e}")

# ============================================================
# Truy vấn SQL
# ============================================================
property_id = result_df['propertyDetails_propertyId'].sample(n=1).iloc[0]
query = f"""
SELECT propertyDetails_propertyId, url, latitude, longitude
FROM read_parquet({parquet_files})
WHERE propertyDetails_propertyId = '{property_id}'
"""
filtered = duckdb.sql(query).df()
print(f"🔍 Dữ liệu của propertyDetails_propertyId = {property_id}:")
print(filtered)
print()


🔍 Đang truy vấn DuckDB với danh sách file cụ thể...
✅ Truy vấn thành công!
Tìm thấy 142 dòng.

--- 5 dòng đầu tiên ---
  propertyDetails_propertyId
0           1c1d31b9d17dc247
1           1c1d31b9d17dc247
2           a42e495ae164e98f
3           a42e495ae164e98f
4           31a9dc6a11b8c174
🔍 Dữ liệu của propertyDetails_propertyId = 013866742ef0bbbc:
  propertyDetails_propertyId  \
0           013866742ef0bbbc   
1           013866742ef0bbbc   

                                                 url  latitude  longitude  
0  https://nhadat247.com.vn/ban-nha-2-tang-dep-ho...  16.01646  108.18846  
1  https://nhadat247.com.vn/ban-nha-2-tang-dep-ho...  16.01646  108.18846  



In [5]:
# ============================================================
# Truy vấn SQL tương thích với schema mới (8 cột)
# ============================================================
property_id = result_df['propertyDetails_propertyId'].sample(n=1).iloc[0]
query = f"""
SELECT propertyDetails_propertyId, "Mức giá", "Diện tích", "Địa chỉ", url, latitude, longitude
FROM read_parquet({parquet_files})
WHERE propertyDetails_propertyId = '{property_id}'
"""
filtered = duckdb.sql(query).df()
print(f"🔍 Dữ liệu của propertyDetails_propertyId = {property_id}:")
print(filtered)
print()

🔍 Dữ liệu của propertyDetails_propertyId = d437663fbb28a3b1:
  propertyDetails_propertyId  Mức giá Diện tích  \
0           d437663fbb28a3b1  12.3 Tỷ      34m2   
1           d437663fbb28a3b1  12.3 Tỷ      34m2   
2           d437663fbb28a3b1  12.3 Tỷ      34m2   
3           d437663fbb28a3b1  12.3 Tỷ      34m2   
4           d437663fbb28a3b1  12.3 Tỷ      34m2   
5           d437663fbb28a3b1  12.3 Tỷ      34m2   

                                Địa chỉ  \
0  Phường Khương Đình,Thanh Xuân,Hà Nội   
1  Phường Khương Đình,Thanh Xuân,Hà Nội   
2  Phường Khương Đình,Thanh Xuân,Hà Nội   
3  Phường Khương Đình,Thanh Xuân,Hà Nội   
4  Phường Khương Đình,Thanh Xuân,Hà Nội   
5  Phường Khương Đình,Thanh Xuân,Hà Nội   

                                                 url  latitude  longitude  
0  https://nhadat247.com.vn/sieu-pham-mat-pho-bui...  20.99821  105.81379  
1  https://nhadat247.com.vn/sieu-pham-mat-pho-bui...  20.99821  105.81379  
2  https://nhadat247.com.vn/sieu-pham-mat-pho-bui..

In [6]:
# ============================================================
# Truy vấn SQL tương thích với schema mới (8 cột)
# ============================================================
import pandas as pd

# Hiển thị đầy đủ nội dung các cột
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 0)

property_id = result_df['propertyDetails_propertyId'].sample(n=1).iloc[0]

query = f"""
SELECT propertyDetails_propertyId,
       "Mức giá",
       "Diện tích",
       "Địa chỉ",
       "Tiêu đề",
       url,
       latitude,
       longitude
FROM read_parquet({parquet_files})
WHERE propertyDetails_propertyId = '{property_id}'
"""

filtered = duckdb.sql(query).df()

print(f"🔍 Dữ liệu chi tiết của propertyDetails_propertyId = {property_id}:")
print("=" * 120)
print(filtered.to_string(index=False))   # In toàn bộ nội dung từng cột không bị rút gọn
print("=" * 120)
print(f"✅ Tổng số dòng: {len(filtered)}")
print()


🔍 Dữ liệu chi tiết của propertyDetails_propertyId = 4cb28fc5d1692133:
propertyDetails_propertyId Mức giá Diện tích                                               Địa chỉ                                                Tiêu đề                                                                                   url  latitude  longitude
          4cb28fc5d1692133   79 TỶ     100m² Đường Nguyễn Phú Hường,Phường Hòa Thọ Tây,Quận Cẩm Lệ 🌟 ĐẤT KIỆT Ô TÔ HÒA THỌ TÂY – 100m² – GIÁ CHỈ 1,79 TỶ! https://nhadat247.com.vn/dat-kiet-o-to-hoa-tho-tay-100m-gia-chi-1-79-ty-pid99351.html  16.01646  108.18846
          4cb28fc5d1692133   79 TỶ     100m² Đường Nguyễn Phú Hường,Phường Hòa Thọ Tây,Quận Cẩm Lệ 🌟 ĐẤT KIỆT Ô TÔ HÒA THỌ TÂY – 100m² – GIÁ CHỈ 1,79 TỶ! https://nhadat247.com.vn/dat-kiet-o-to-hoa-tho-tay-100m-gia-chi-1-79-ty-pid99351.html  16.01646  108.18846
✅ Tổng số dòng: 2



In [7]:
# ============================================================
# DESCRIBE HISTORY (Lịch sử Delta)
# ============================================================
print("🕓 DESCRIBE HISTORY (Lịch sử thay đổi bảng):")
try:
    history = dt.history()
    if history:
        for h in history:
            print(f"Version {h['version']}: {h['timestamp']} - {h['operation']}")
    else:
        print("⚠️ Không có thông tin lịch sử (Delta log trống).")
except Exception as e:
    print(f"⚠️ Lỗi khi đọc lịch sử: {e}")
print()


🕓 DESCRIBE HISTORY (Lịch sử thay đổi bảng):
Version 5: 1762940920468 - MERGE
Version 4: 1762866568088 - MERGE
Version 3: 1762866070414 - MERGE
Version 2: 1762865823689 - MERGE
Version 1: 1762865525717 - MERGE
Version 0: 1762865074576 - WRITE



In [8]:
# ============================================================
# Đọc dữ liệu qua DuckDB
# ============================================================
print("📦 Đang đọc dữ liệu từ MinIO (DuckDB)...")

try:
    df = duckdb.sql(f"SELECT * FROM read_parquet({parquet_files})").df()
    print(f"✅ Đọc thành công {len(df)} dòng dữ liệu.")
    print("📋 Các cột có sẵn:")
    print(df.columns.tolist())

    # --- Thống kê mô tả ---
    print("\n📊 Thống kê mô tả:")
    print(df.describe(include='all').transpose())
except Exception as e:
    print(f"❌ Lỗi khi đọc Parquet từ MinIO: {e}")
    df = pd.DataFrame()
print()

📦 Đang đọc dữ liệu từ MinIO (DuckDB)...
✅ Đọc thành công 142 dòng dữ liệu.
📋 Các cột có sẵn:
['url', 'Diện tích', 'Mức giá', 'Địa chỉ', 'Ngày đăng', 'Tiêu đề', 'latitude', 'longitude', 'search_city', 'search_property_type', 'search_rent_or_buy', 'search_radius', 'propertyDetails_propertyId']

📊 Thống kê mô tả:
                            count unique  \
url                           142     36   
Diện tích                     142     18   
Mức giá                       142     25   
Địa chỉ                       142     31   
Ngày đăng                     142      2   
Tiêu đề                       142     36   
latitude                    142.0    NaN   
longitude                   142.0    NaN   
search_city                   142      4   
search_property_type          142      1   
search_rent_or_buy            142      1   
search_radius               142.0    NaN   
propertyDetails_propertyId    142     36   

                                                                       

In [9]:
# ============================================================
# SELECT COUNT(*)
# ============================================================
if not df.empty:
    print("📈 SELECT COUNT(*) FROM delta:")
    count_result = duckdb.sql(f"SELECT COUNT(*) AS total_rows FROM read_parquet({parquet_files})").df()
    print(count_result)
    print()

    # ============================================================
    # DESCRIBE bảng
    # ============================================================
    print("📋 DESCRIBE (Cấu trúc cơ bản):")
    duckdb.sql(f"DESCRIBE SELECT * FROM read_parquet({parquet_files})").show()
    print()

    # ============================================================
    # 💾 Ghi ra CSV (tùy chọn)
    # ============================================================
    # df.to_csv("property_data_duckdb.csv", index=False, encoding="utf-8")
    # print("💾 Đã lưu dữ liệu ra file: property_data_duckdb.csv")

else:
    print("⚠️ Không có dữ liệu để xử lý.")

📈 SELECT COUNT(*) FROM delta:
   total_rows
0         142

📋 DESCRIBE (Cấu trúc cơ bản):
┌────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│        column_name         │ column_type │  null   │   key   │ default │  extra  │
│          varchar           │   varchar   │ varchar │ varchar │ varchar │ varchar │
├────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ url                        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Diện tích                  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Mức giá                    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Địa chỉ                    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Ngày đăng                  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Tiêu đề                    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ latitude                   │ DOUBLE      │ YES     │ NULL  

In [10]:
# from nbconvert import WebPDFExporter
# from traitlets.config import Config
# import nbformat

# notebook_path = "E:/Real-Estate_Project_Data_Engineering/practical-data-engineering/src/pipelines/real-estate/realestate/notebooks/TEST.ipynb"

# with open(notebook_path, "r", encoding="utf-8") as f:
#     nb_node = nbformat.read(f, as_version=4)

# c = Config()
# c.WebPDFExporter.allow_chromium_download = True

# exporter = WebPDFExporter(config=c)

# body, _ = exporter.from_notebook_node(nb_node)

# with open("TEST.pdf", "wb") as f:
#     f.write(body)

# print("✅ Đã xuất thành công file TEST.pdf")

✅ Đã xuất thành công file TEST.pdf
